In [1]:
import pandas as pd
import numpy as np

In [2]:
ppr = 1.0

In [3]:
df_raw = pd.read_csv('data/NFL Play by Play 2009-2018 (v5).csv')

/Users/high5games/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_raw['subtract'] = (df_raw['game_id'].astype(str).str[4:6].astype(int) < 7).astype(int)
df_raw['season'] = df_raw['game_id'].astype(str).str[:4].astype(int) - df_raw['subtract']
df_raw = df_raw.drop('subtract',1)

In [5]:
# aggregate passing stats
df_pass = df_raw[['season','passer_player_name','passer_player_id','yards_gained','interception','pass_touchdown'
                  ]].loc[(df_raw['sack']!=1) & (df_raw['two_point_attempt']!=1)]
df_pass = df_pass.groupby(['season','passer_player_name','passer_player_id']).sum().reset_index()
df_pass.columns = ['Season','Player','PlayerID','PassYards','Interceptions','PassTDs']

In [6]:
# aggregate rushing
df_rush = df_raw[['season','rusher_player_name','rusher_player_id','yards_gained','rush_touchdown'
                 ]].loc[df_raw['two_point_attempt']!=1]
df_rush = df_rush.groupby(['season','rusher_player_name','rusher_player_id']).sum().reset_index()
df_rush.columns = ['Season','Player','PlayerID','RushYds','RushTDs']

In [7]:
# aggregate receiving
df_receive = df_raw[['season','receiver_player_name','receiver_player_id','yards_gained','pass_touchdown','play_id'
                 ]].loc[df_raw['two_point_attempt']!=1]
df_receive = df_receive.groupby(['season','receiver_player_name','receiver_player_id']).agg({
    'yards_gained' : sum,
    'pass_touchdown' : sum,
    'play_id' : 'count'
}).reset_index()
df_receive.columns = ['Season','Player','PlayerID','ReceivingYds','ReceivingTDs','Receptions']

In [8]:
# 2-point conversion passing stats
df_2pt_pass = df_raw[['season','passer_player_name',
                      'passer_player_id','play_id']].loc[df_raw['two_point_conv_result']=='success']
df_2pt_pass = df_2pt_pass.groupby(['season','passer_player_name','passer_player_id']).count().reset_index()
df_2pt_pass.columns = ['Season','Player','PlayerID','Pass2Pt']

In [9]:
# 2-point conversion rushing stats
df_2pt_rush = df_raw[['season','rusher_player_name',
                      'rusher_player_id','play_id']].loc[df_raw['two_point_conv_result']=='success']
df_2pt_rush = df_2pt_rush.groupby(['season','rusher_player_name','rusher_player_id']).count().reset_index()
df_2pt_rush.columns = ['Season','Player','PlayerID','Rush2Pt']

In [10]:
# 2-point conversion rushing stats
df_2pt_receive = df_raw[['season','receiver_player_name',
                      'receiver_player_id','play_id']].loc[df_raw['two_point_conv_result']=='success']
df_2pt_receive = df_2pt_receive.groupby(['season','receiver_player_name','receiver_player_id']).count().reset_index()
df_2pt_receive.columns = ['Season','Player','PlayerID','Receive2Pt']

In [11]:
# fumbles
df_fumble = df_raw[['season','fumbled_1_player_name','fumbled_1_player_id','play_id'
                 ]].loc[(df_raw['two_point_attempt']!=1) & (df_raw['fumble_lost']==1)]
df_fumble = df_fumble.groupby(['season','fumbled_1_player_name','fumbled_1_player_id']).count().reset_index()
df_fumble.columns = ['Season','Player','PlayerID','FumblesLost']

In [12]:
df_out = df_pass.merge(df_rush, on=['Season','Player','PlayerID'], how='outer')
df_out = df_out.merge(df_receive, on=['Season','Player','PlayerID'], how='outer')
df_out = df_out.merge(df_2pt_pass, on=['Season','Player','PlayerID'], how='outer')
df_out = df_out.merge(df_2pt_rush, on=['Season','Player','PlayerID'], how='outer')
df_out = df_out.merge(df_2pt_receive, on=['Season','Player','PlayerID'], how='outer')
df_out = df_out.merge(df_fumble, on=['Season','Player','PlayerID'], how='outer')
df_out = df_out.fillna(0.)
df_out.sort_values(by=['Season','Player']).head()

,Season,Player,PlayerID,PassYards,Interceptions,PassTDs,RushYds,RushTDs,ReceivingYds,ReceivingTDs,Receptions,Pass2Pt,Rush2Pt,Receive2Pt,FumblesLost
3332,2009,A.Battle,00-0022064,0.0,0.0,0.0,0.0,0.0,40.0,0.0,5.0,0.0,0.0,0.0,2.0
3333,2009,A.Becht,00-0019457,0.0,0.0,0.0,0.0,0.0,61.0,1.0,12.0,0.0,0.0,0.0,1.0
956,2009,A.Boldin,00-0022084,0.0,0.0,0.0,7.0,0.0,1029.0,5.0,128.0,0.0,0.0,0.0,2.0
957,2009,A.Bradshaw,00-0025637,0.0,0.0,0.0,778.0,7.0,207.0,0.0,30.0,0.0,0.0,0.0,2.0
958,2009,A.Brown,00-0027129,0.0,0.0,0.0,131.0,0.0,84.0,1.0,15.0,0.0,0.0,0.0,0.0


In [13]:
df_out['FPts'] = df_out['PassYards']*0.04 + df_out['Interceptions']*-2 + df_out['PassTDs']*4 + \
    df_out['RushYds']*0.1 + df_out['RushTDs']*6 + df_out['ReceivingYds']*0.1 + df_out['ReceivingTDs']*6 + \
    df_out['Receptions']*ppr + df_out['Pass2Pt']*2 + df_out['Rush2Pt']*2 + df_out['Receive2Pt']*2 + \
    df_out['FumblesLost']*-2
df_out.sort_values(by=['Season','Player']).head()

,Season,Player,PlayerID,PassYards,Interceptions,PassTDs,RushYds,RushTDs,ReceivingYds,ReceivingTDs,Receptions,Pass2Pt,Rush2Pt,Receive2Pt,FumblesLost,FPts
3332,2009,A.Battle,00-0022064,0.0,0.0,0.0,0.0,0.0,40.0,0.0,5.0,0.0,0.0,0.0,2.0,5.0
3333,2009,A.Becht,00-0019457,0.0,0.0,0.0,0.0,0.0,61.0,1.0,12.0,0.0,0.0,0.0,1.0,22.1
956,2009,A.Boldin,00-0022084,0.0,0.0,0.0,7.0,0.0,1029.0,5.0,128.0,0.0,0.0,0.0,2.0,257.6
957,2009,A.Bradshaw,00-0025637,0.0,0.0,0.0,778.0,7.0,207.0,0.0,30.0,0.0,0.0,0.0,2.0,166.5
958,2009,A.Brown,00-0027129,0.0,0.0,0.0,131.0,0.0,84.0,1.0,15.0,0.0,0.0,0.0,0.0,42.5


In [14]:
df_out[['Season','Player','PlayerID','FPts']].to_csv('data/seasonTotals.csv', index=False)